In [ ]:
import sys, os
sys.path.append(os.getcwd())
%cd ../

/


In [ ]:
import torch
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving social_media_comment.csv to social_media_comment (1).csv


In [ ]:
import pandas as pd
import numpy as np


comment_text = pd.read_csv('social_media_comment.csv')

df = comment_text.copy()

In [ ]:
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

class BertForMultilabelSequenceClassification(BertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)

    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict)

        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = torch.nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.num_labels),
                            labels.float().view(-1, self.num_labels))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set the path to your directory
path = '/content/drive/My Drive/model_output/'

# List all files in the directory
print(os.listdir(path))

['model.safetensors', 'training_args.bin', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.txt', 'tokenizer.json', 'config.json']


In [ ]:
# Load Configuration and Model
from transformers import BertConfig, BertForSequenceClassification

# Load the configuration file
config_path = os.path.join(path, 'config.json')
config = BertConfig.from_json_file(config_path)

# Load the model
model_path = os.path.join(path, 'model.safetensors')
emotion_model = BertForSequenceClassification.from_pretrained(model_path, config=config, use_safetensors=True)

In [ ]:
from transformers import BertConfig, AutoTokenizer

In [ ]:
from scipy.special import softmax

MODEL_NAME = 'hfl/chinese-roberta-wwm-ext'
labels = ['anger','anxiety','expect','hate','joy','love','sorrow','surprise','neutral']
NUM_LABELS =  len(labels)

emotion_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
emotion_model = emotion_model.cpu()

In [ ]:
def predict_and_append(seqs_to_classify, df, labels):
    for seq in seqs_to_classify:
        # Check if 'seq' (comment in list of comments) is a non-empty string
        if isinstance(seq, str) and len(seq) > 0:
            # Tokenize the cleaned input text (single example) using the tokenizer
            encoded_input = emotion_tokenizer(seq, return_tensors='pt', truncation=True, padding="max_length", max_length=512)

            # Make predictions using the loaded model;  generated by the emotion_tokenizer, include input_ids, attention_mask, and token_type_ids
            output = emotion_model(**encoded_input)

            # Extract the prediction scores
            scores = output.logits.sigmoid() # get probabilities
            scores = scores[0]  # Get the predictions arrary for the instance

            # Create a dictionary to store the prediction results for each emotion label
            predictions_dict = {f'oritext_{label}': 0.0 for label in labels}
            ranking = scores.argsort(descending=True)
            for i in range(scores.shape[0]):
                l = labels[ranking[i]]
                s = scores[ranking[i]]
                predictions_dict[f'oritext_{l}'] = np.round(float(s), 4)
            # Append the predictions to the corresponding row in the DataFrame
            idx = df[df['oritextall_cleaned2'] == seq].index[0]
            for label, score in predictions_dict.items():
                df.at[idx, label] = score
        else:
            print(f"Skipping empty or invalid text: {seq}")

In [ ]:
# Alternative Model Specification
  # seqs_to_classify = list(zip(sub_text['oritextall_cleaned2'], sub_text['_id']))

  # for seq, comment_id in seqs_to_classify:

  # if comment_id in df['_id'].values:
  #               idx = df[df['_id'] == comment_id].index[0]
  #               for label, score in predictions_dict.items():
  #                   df.at[idx, label] = score

In [ ]:
df.shape

(500, 5)

In [ ]:
df.shape[0]

500

In [ ]:
# Remove comments that is null after preprocessing
sub_text = df[~df['oritextall_cleaned2'].isnull()]
sub_text.shape

(462, 5)

In [ ]:
sub_emo = df[~df['emoji_textall_cleaned'].isnull()]
sub_emo.shape

(111, 5)

In [ ]:
sub_text[sub_text['oritextall_cleaned2'].str.contains("nan")].head(2)

,Unnamed: 0.1,Unnamed: 0,_id,oritextall_cleaned2,emoji_textall_cleaned


In [ ]:
#######################Process Text in Comments #########################
sub_text = sub_text.reset_index()
seqs_to_classify = sub_text['oritextall_cleaned2'].tolist()
predict_and_append(seqs_to_classify, sub_text, labels)

In [ ]:
sub_text[~sub_text['oritextall_cleaned2'].isnull()].head(2)

,index,Unnamed: 0.1,Unnamed: 0,_id,oritextall_cleaned2,emoji_textall_cleaned,oritext_anger,oritext_anxiety,oritext_expect,oritext_hate,oritext_joy,oritext_love,oritext_sorrow,oritext_surprise,oritext_neutral
0,0,0,0,C_3698509578700451,我爱豆西装配皮鞋,NaN,0.0034,0.0060,0.0065,0.0028,0.0324,0.6890,0.0078,0.0035,0.0165
1,4,4,4,C_3697733028468105,凑人数!。。。干嘛不叫我去啊，哈哈哈哈,NaN,0.0140,0.0198,0.1599,0.0100,0.6621,0.0798,0.0058,0.0044,0.0039


In [ ]:
def emo_predict_and_append(seqs_to_classify, df, labels):
    for seq in seqs_to_classify:
        # Skip if the sequence is NaN
        if pd.notna(seq):
            # To predict, we simply tokenize the input text and feed it into our model
            encoded_input = emotion_tokenizer(seq, return_tensors='pt', truncation=True, padding=True)
            output = emotion_model(**encoded_input)

            scores = output[0][0].detach().sigmoid().numpy()
            ranking = np.argsort(scores)
            ranking = ranking[::-1]
            # Create a dictionary to store the prediction results for each emotion label; difference emo sequence for each comment, updated with highest scoring emotion first
            predictions_dict = {f'emotext_{label}': 0.0 for label in labels}
            for i in range(scores.shape[0]):
                l = labels[ranking[i]]
                s = scores[ranking[i]]
                predictions_dict[f'emotext_{l}'] = np.round(float(s), 4)
            # Append the predictions to the corresponding row in the DataFrame
            idx = df[df['emoji_textall_cleaned'] == seq].index[0]
            for label, score in predictions_dict.items():
                df.at[idx, label] = score

In [ ]:
sub_emo= sub_emo.reset_index()

In [ ]:
sub_emo.head(2)

,index,Unnamed: 0.1,Unnamed: 0,_id,oritextall_cleaned2,emoji_textall_cleaned
0,2,2,2,C_3697737440333373,NaN,爱你
1,6,6,6,C_3697893830022996,回复@四毛et毛毛-赛巴斯缇安版机密:胖点好〜哈哈哈男孩子嘛〜〜〜,哈哈 哈哈 哈哈


In [ ]:
seqs_to_classify = sub_emo['emoji_textall_cleaned'].tolist()
emo_predict_and_append(seqs_to_classify, sub_emo, labels)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
sub_emo.head(2)

,index,Unnamed: 0.1,Unnamed: 0,_id,oritextall_cleaned2,emoji_textall_cleaned,emotext_anger,emotext_anxiety,emotext_expect,emotext_hate,emotext_joy,emotext_love,emotext_sorrow,emotext_surprise,emotext_neutral
0,2,2,2,C_3697737440333373,NaN,爱你,0.0031,0.0152,0.0078,0.0041,0.0220,0.7102,0.014,0.0023,0.0104
1,6,6,6,C_3697893830022996,回复@四毛et毛毛-赛巴斯缇安版机密:胖点好〜哈哈哈男孩子嘛〜〜〜,哈哈 哈哈 哈哈,0.0082,0.0126,0.0370,0.0091,0.7573,0.0655,0.009,0.0047,0.0069


In [ ]:
sub_text.to_csv('subcomemnts_textscore_demo.csv')
files.download('subcomemnts_textscore_demo.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sub_emo.to_csv('subcomemnts_emoscore_demo.csv')
files.download('subcomemnts_emoscore_demo.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>